In [1]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
import numpy as np
import torch.nn as nn
import torch
import os
from sklearn.metrics import f1_score
import random
from torch.nn import functional as F
import torch.optim as optim
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Importing dataset 

In [0]:
data_folder = '/content/drive/My Drive/NLP_HMW2'
train_data = pd.read_csv("/content/drive/My Drive/NLP_HMW2/train.csv", encoding="ISO-8859-1")
dev_data = pd.read_csv("/content/drive/My Drive/NLP_HMW2/dev.csv", encoding="ISO-8859-1")
test_data = pd.read_csv("/content/drive/My Drive/NLP_HMW2/test_no_label.csv", encoding="ISO-8859-1")


In [3]:
train_set = [val for key, val in train_data.items()]
dev_set = [val for key, val in dev_data.items()]
test_set = [val for key, val in test_data.items()]

len(train_set), len(dev_set), len(test_set)

(3, 3, 2)

In [0]:
for i in range(len(train_data)):
    if len(train_set[1][i]) < 1000:
        print('ID: {} \nComment: {} \nLabel: {}\n'.format(i, train_set[1][i], train_set[2][i]))

# Tokenize

In [0]:
tokenizer = TweetTokenizer()


def tokenization(data):
    for i in range(len(data[1])):
        tokens = tokenizer.tokenize(data[1][i].lower())
        data[1][i] = tokens
    return data


train_set = tokenization(train_set)
dev_set = tokenization(dev_set)
test_set = tokenization(test_set)

train_token_list = []
dev_token_list = []
test_token_list = []
for i in range(len(train_set[1])):
    train_token_list.append(train_set[1][i])
for i in range(len(dev_set[1])):
    dev_token_list.append(dev_set[1][i])
for i in range(len(test_set[1])):
    test_token_list.append(test_set[1][i])

In [6]:
train_set_=list(zip(train_token_list, train_set[2]))
train_set_[2]

(['now',
  'question',
  'is',
  ',',
  'pakistan',
  'will',
  'adhere',
  'to',
  'this',
  '?'],
 'OAG')

In [0]:
dev_set_=list(zip(dev_token_list, dev_set[2]))
for (token,label) in dev_set_:
    print(token,label)

# Vocabulary

In [51]:
vocabulary = set([t for tokens in train_token_list + dev_token_list for t in tokens])
print('Vocabulary size: {}'.format(len(vocabulary)))
vocab2idx = {w: i+2 for i, w in enumerate(vocabulary)}
idx2vocab = {i+2: w for i, w in enumerate(vocabulary)}

vocab2idx['<PAD>'] = 0
vocab2idx['<UNK>'] = 1

idx2vocab[0] = '<PAD>'
idx2vocab[1] = '<UNK>'

print('Vocabulary size: {}'.format(len(vocab2idx)))


Vocabulary size: 25661
Vocabulary size: 25663


# Token Encoder

In [0]:
class WordEncoder(nn.Module):
    def __init__(self, vocab, seqlen=300, varlen = False):
        super(WordEncoder, self).__init__()

        self.word2idx = vocab
        self.sequence_length = seqlen
        self.various_length = varlen

    def one_hot(self, token_id):

        encoded = np.zeros(len(self.word2idx), dtype=int)
        encoded[token_id] = 1.0
        return encoded

    def forward(self, samples):
        pad_idx = self.word2idx['<PAD>']
        unk_idx = self.word2idx['<UNK>']

        if self.various_length:

            # Find the length of the longest sample
            maxlen = max([len(s) for s in samples])

        else:

            # Use a constant length for all samples
            maxlen = self.sequence_length

        encoded = [[self.word2idx.get(token, unk_idx) for token in tokens] for tokens in samples]

        padded = np.zeros((len(samples), maxlen), dtype=int)
        masks = torch.zeros(len(samples), maxlen).long()
        # padding and Masking
        for i in range(len(encoded)):
            padded[i, :len(encoded[i])] = np.array(encoded[i])[:maxlen]
            masks[i, :len(encoded[i])] = 1

        encoded = [[self.one_hot(token) for token in tokens] for tokens in padded]
        encoded = torch.tensor(encoded).long()

        if torch.cuda.is_available():
            encoded = encoded.cuda()
            masks = masks.cuda()

        result = {
            'mask': masks,
            'encoded': encoded
        }

        return result

In [10]:
encoder1 = WordEncoder(vocab2idx)
encoder2 = WordEncoder(vocab2idx, varlen=True)
tokens = train_token_list[4984-1:4984+1]
encoder_result = encoder1(tokens)

print("Encoded:\n{}".format(encoder_result['encoded']))
print("Encoding Shape:\n{}\n".format(encoder_result['encoded'].shape))
print("Mask:\n{}".format(encoder_result['mask']))
# print("Labels:\n{}\n".format(labels))
print("Tokens:\n{}".format(tokens))

tokens = train_token_list[4984-1:4984+1]
encoder_result = encoder2(tokens)

print("Encoding Shape:\n{}\n".format(encoder_result['encoded'].shape))

Encoded:
tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0]]])
Encoding Shape:
torch.Size([2, 300, 25663])

Mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  

In [0]:
class WordEmbedder(nn.Module):
    def __init__(self, vocab, glove_file, seqlen=300, varlen=False):
        super(WordEmbedder, self).__init__()
        assert os.path.exists(glove_file) and glove_file.endswith('.txt'), glove_file

        self.emb_dim = None

        self.PAD_TOKEN = '<PAD>'
        self.UNK_TOKEN = '<UNK>'
        self.sequence_length = seqlen
        self.various_length = varlen

        idx2word = [self.PAD_TOKEN, self.UNK_TOKEN]
        idx2vect = [None, None]

        with open(glove_file, 'r', encoding="utf8") as fp:
            for line in fp:
                line = line.split()

                if line[0] not in vocab:
                    continue

                w = line[0]
                v = np.array([float(value) for value in line[1:]])

                if self.emb_dim is None:
                    self.emb_dim = v.shape[0]

                idx2word.append(w)
                idx2vect.append(v)

        idx2vect[0] = np.zeros(self.emb_dim)
        idx2vect[1] = np.mean(idx2vect[2:], axis=0)

        self.embeddings = torch.from_numpy(np.array(idx2vect)).float()
        self.embeddings = nn.Embedding.from_pretrained(self.embeddings, freeze=False)

        self.idx2word = {i: w for i, w in enumerate(idx2word)}
        self.word2idx = {w: i for i, w in self.idx2word.items()}

    def forward(self, samples):
        pad_idx = self.word2idx[self.PAD_TOKEN]
        unk_idx = self.word2idx[self.UNK_TOKEN]

        if self.various_length:

            # Find the length of the longest sample
            maxlen = max([len(s) for s in samples])

        else:

            # Use a constant length for all samples
            maxlen = self.sequence_length

        encoded = [[self.word2idx.get(token, unk_idx) for token in tokens] for tokens in samples]

        padded = np.zeros((len(samples), maxlen), dtype=int)
        masks = torch.zeros(len(samples), maxlen).long()
         # Padding and masking
        for i in range(len(encoded)):
            masks[i, :len(encoded[i])] = 1
            padded[i, :len(encoded[i])] = np.array(encoded[i])[:maxlen]
            # encoded[i] += [pad_idx] * max(0, (maxlen - len(encoded[i])))

        encoded = torch.tensor(padded).long()

        if torch.cuda.is_available():
            encoded = encoded.cuda()
            masks = masks.cuda()

        result = {
            'output': self.embeddings(encoded),
            'mask': masks,
            'encoded': encoded
        }

        return result



In [12]:
embedder = WordEmbedder(vocabulary, '/content/drive/My Drive/NLP_HMW2/glove.6B.100d.txt', varlen=True)

tokens = train_token_list[4984-1:4984+1]
embedder_result = embedder(tokens)

print("Encoded:\n{}".format(embedder_result['encoded']))
print("Embedding Shape:\n{}\n".format(embedder_result['output'].shape))
print("Tokens:\n{}".format(tokens))

Encoded:
tensor([[  579,   185,     1,    35,     2,    47, 12371,    28,   235,  1004,
            63,     2,   107, 10859,   418,    79, 13673,     4,    62,    15,
             9,  1208,   724,  2501,    32,    50,  8164,   958,   294,     7,
            36,    15,   545,   167,   858,     5,    43, 10085,     4],
        [ 4270, 15341,     1,   187,  1005,   429,  1058,    23,   160,   429,
          7268,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])
Embedding Shape:
torch.Size([2, 39, 100])

Tokens:
[['really', '?', "i'm", 'not', 'the', 'one', 'dictating', 'an', 'official', 'language', 'all', 'the', 'while', 'degrading', 'others', 'you', 'hypocrite', '.', 'there', 'is', 'a', 'reason', 'why', 'jews', 'have', 'been', 'hated', 'throughout', 'history', 'and', 'this', 'is', 'yet', 'another', 'example', 'of', 'their', 

# BiGRU Layer

In [0]:
class BiGRULayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(BiGRULayer, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.gru = nn.GRU(input_dim, hidden_dim // 2, bidirectional=True)

    def forward(self, vectors, mask):
        batch_size = vectors.size(0)
        maxlen = vectors.size(1)
        lengths = mask.sum(-1)
        
        gru_out, _ = self.gru(vectors)  # (batch, seq_len, hidden_dim)
        
        assert gru_out.size(0) == batch_size
        assert gru_out.size(1) == maxlen
        assert gru_out.size(2) == self.hidden_dim

        # Separate the directions of the GRU
        gru_out = gru_out.view(batch_size, maxlen, 2, self.hidden_dim // 2)

        # Pick up the last hidden state per direction
        fw_last_hn = gru_out[range(batch_size), lengths - 1, 0]  # (batch, hidden // 2)
        bw_last_hn = gru_out[range(batch_size), 0, 1]            # (batch, hidden // 2)
        
        last_hn = torch.cat([fw_last_hn, bw_last_hn], dim=1)      # (batch, hidden // 2) -> (batch, hidden)

        return {'output': last_hn, 'outputs': gru_out}


bigru_layer = BiGRULayer(embedder.emb_dim, 60)

In [14]:
bigru_layer

BiGRULayer(
  (gru): GRU(100, 30, bidirectional=True)
)

# RNN Classifier

In [0]:
class RNNClassifier(nn.Module):
    def __init__(self, embedder, extractor, num_class):
        super(RNNClassifier, self).__init__()
        self.embedder = embedder
        self.extractor = extractor
        self.classifier = nn.Linear(extractor.hidden_dim, num_class)

    def forward(self, tokens, targets=None):
        embedded = self.embedder(tokens)
        extracted = self.extractor(embedded['output'], embedded['mask'])

        output = torch.nn.functional.softmax(self.classifier(extracted['output']), dim=1)

        return output


gru_model = RNNClassifier(embedder, bigru_layer, num_class=3)

In [16]:
dummy_tokens = ['you are stupid'.split(), 'you are nice!'.split(), 'the deeper you dive the shallower cushion you have.'.split()]
dummy_labels = torch.tensor([0, 1]).long()
result = gru_model(dummy_tokens, dummy_labels)

probs = result.detach().cpu().numpy().tolist()
preds = result.detach().cpu().numpy().tolist()
for i in range(len(preds)):
    for j in range(len(preds[i])):
        if preds[i][j] == max(preds[i]):
            preds[i][j] = 1
        else:
            preds[i][j] = 0

print('predictions: {}\nprobabilities: {}'.format(preds, probs))

predictions: [[0, 1, 0], [0, 1, 0], [0, 1, 0]]
probabilities: [[0.3167872726917267, 0.36374107003211975, 0.31947168707847595], [0.2993258535861969, 0.3759068548679352, 0.3247673511505127], [0.27169036865234375, 0.38944023847579956, 0.33886945247650146]]


In [0]:
def track_best_model(model_path, model, epoch, best_acc, dev_acc, dev_loss):
    if best_acc > dev_acc:
        return best_acc, ''
    state = {
        'epoch': epoch,
        'F1_Score': dev_acc,
        'loss': dev_loss,
        'model': model.state_dict()
    }
    torch.save(state, model_path)
    
    return dev_acc, ' * '

# Training the model:

In [0]:
def train(model, optimizer, shuffled_train_set, batch_size):
    model.train()
    total_loss = 0
    batch_tokens, batch_target = [], []
    random.Random(1234).shuffle(shuffled_train_set)
    
    for i in range(len(shuffled_train_set)):
      (token, label) = shuffled_train_set[i]
      batch_tokens.append(token)
      if label == 'NAG':
        label_ = [1, 0, 0]
      elif label == 'CAG':
        label_ = [0, 1, 0]
      elif label == 'OAG':
        label_ = [0, 0, 1]
      batch_target.append(label_)
      if len(batch_tokens) == batch_size or i == len(shuffled_train_set) - 1:
        optimizer.zero_grad()
        out = model(batch_tokens)
        y_pred = out.cpu()
        # print(len(y_pred))
        # print(len(batch_target))
        # exit()
        loss = F.binary_cross_entropy(y_pred, torch.tensor(batch_target).float())
        loss.backward()
        optimizer.step()
        batch_tokens, batch_target = [], []
    return model, shuffled_train_set

# Evaluate

In [0]:
idx2label = {'NAG':0, 'CAG':1, 'OAG':2}
def evaluate(model, dev_set_, batch_size):
    model.eval()
    total_loss = 0
    batch_tokens, batch_target = [], []
    predictions, actual = [], []

    with torch.no_grad():
        for i in range(len(dev_set_)):
            (token, label) = dev_set_[i]
            batch_tokens.append(token)
            if label == 'NAG':
                label_ = [1, 0, 0]
            elif label == 'CAG':
                label_ = [0, 1, 0]
            elif label == 'OAG':
                label_ = [0, 0, 1]
            batch_target.append(label_)
            actual.append(idx2label[label])
            if len(batch_tokens) == batch_size or i == len(dev_set_) - 1:
                  out = model(batch_tokens,batch_target)
                  y_pred = out.cpu()
                  predictions.extend(list(torch.argmax(y_pred, -1).numpy()))
                  # actual.extend(batch_target)
                  loss = F.binary_cross_entropy(y_pred, torch.tensor(batch_target).float())
                  total_loss += loss.item()
                  batch_tokens, batch_target = [], []
    
    f1_score_ = f1_score(actual, predictions, average='weighted')
    return predictions, total_loss / len(dev_set_), f1_score_


In [0]:
def training_loop(config, model_):

    model = model_
    optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=config['momentum'])

    shuffled_train_set = train_set_
    best_acc = 0

    for epoch in range(config['epoch']):

        epoch_msg = '[Epoch {}] / {}'.format(epoch+1, config['epoch'])

        model, shuffled_train_set = train(model, optimizer, shuffled_train_set, batch_size=32)

        train_pred, train_loss, train_acc = evaluate(model, shuffled_train_set, batch_size=128)
        epoch_msg += ' [TRAIN] Loss: {:.4f}, Weighted F1_Score: {:.4f}'.format(train_loss, train_acc)
        val_pred, val_loss, val_acc = evaluate(model, dev_set_, batch_size=128)
        epoch_msg += ' [DEV] Loss: {:.4f}, Weighted F1_Score: {:.4f}'.format(val_loss, val_acc)
        # test_pred, test_loss, test_acc = evaluate(model, test_token_list, batch_size=128)
        # epoch_msg += ' [TEST] Loss: {:.4f}, Weighted F1_Score: {:.4f}'.format(test_loss, test_acc)

        best_acc, epoch_track = track_best_model(config['checkpoint'], model, epoch, best_acc, val_acc, val_loss)
        print(epoch_msg + epoch_track)

    print('Done Training!')

    state = torch.load(config['checkpoint'])
    model.load_state_dict(state['model'])

    print('Returning best model from epoch {} with loss {:.5f} and Weighted F1_Score {:.5f}'.format(state['epoch'], state['loss'], state['F1_Score']))
    return model

In [21]:
config = {
    'lr': 1e-2,
    'momentum': 0.99,
    'epoch': 10,
    'checkpoint': 'rnn_model.pt'
}
model = training_loop(config, gru_model)

[Epoch 1] / 10 [TRAIN] Loss: 0.0047, Weighted F1_Score: 0.4123 [DEV] Loss: 0.0049, Weighted F1_Score: 0.3783 * 
[Epoch 2] / 10 [TRAIN] Loss: 0.0046, Weighted F1_Score: 0.4258 [DEV] Loss: 0.0049, Weighted F1_Score: 0.3961 * 
[Epoch 3] / 10 [TRAIN] Loss: 0.0046, Weighted F1_Score: 0.4700 [DEV] Loss: 0.0048, Weighted F1_Score: 0.4364 * 
[Epoch 4] / 10 [TRAIN] Loss: 0.0045, Weighted F1_Score: 0.4819 [DEV] Loss: 0.0048, Weighted F1_Score: 0.4262
[Epoch 5] / 10 [TRAIN] Loss: 0.0046, Weighted F1_Score: 0.4371 [DEV] Loss: 0.0049, Weighted F1_Score: 0.4043
[Epoch 6] / 10 [TRAIN] Loss: 0.0044, Weighted F1_Score: 0.5076 [DEV] Loss: 0.0048, Weighted F1_Score: 0.4470 * 
[Epoch 7] / 10 [TRAIN] Loss: 0.0044, Weighted F1_Score: 0.5236 [DEV] Loss: 0.0048, Weighted F1_Score: 0.4282
[Epoch 8] / 10 [TRAIN] Loss: 0.0044, Weighted F1_Score: 0.4560 [DEV] Loss: 0.0050, Weighted F1_Score: 0.3802
[Epoch 9] / 10 [TRAIN] Loss: 0.0042, Weighted F1_Score: 0.5341 [DEV] Loss: 0.0049, Weighted F1_Score: 0.4391
[Epoch 

In [22]:
test_labels = torch.tensor([0, 1]).long()
result = model(test_token_list, dummy_labels)

probs_ = result.detach().cpu().numpy().tolist()
preds_ = result.detach().cpu().numpy().tolist()
for i in range(len(preds_)):
    for j in range(len(preds_[i])):
        if preds_[i][j] == max(preds_[i]):
            preds_[i][j] = 1
        else:
            preds_[i][j] = 0

print('predictions: {}\nprobabilities: {}'.format(preds_, probs_))


predictions: [[1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 

In [0]:
test_label = []
for i in range(len(preds_)):
  if preds_[i] == [1, 0, 0]:
    pred = 'NAG'
  elif preds_[i] == [0, 1, 0]:
    pred = 'CAG'
  elif preds_[i]== [0, 0, 1]:
    pred = 'OAG'
  test_label.append(pred)

In [0]:
import csv

with open('test_prediction.csv', 'w') as csvfile:
    fieldnames = ['ID', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames=['ID', 'Label'])

    writer.writeheader()
    for i in range(len(test_set[0])):
      writer.writerow({'ID': test_set[0][i], 'Label': test_label[i]})
  

In [0]:
from google.colab import files
files.download("test_prediction.csv")